In [1]:
import os
import keras
import glob
import numpy as np
from keras.preprocessing.image import  img_to_array, load_img
from PIL import Image
# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
dict_labels = {"test":0}
size = (128, 128)
nbofdata=2000
base_path = r'ml100-03-final/image_data/test/'
layers_of_folders=0
folder_list=[]    
labels=['test']

if base_path :
    folder_layers=[]
    files = os.scandir(base_path)
    #  Get the 1st layer of folder
    first_folder = []
    first_folder_kind = []
    for entry in files:
        if entry.is_dir():
            first_folder.append(entry.path)
            first_folder_kind.append(entry.name)
    folder_layers.append(first_folder_kind)
    folder_list.append(first_folder)

In [3]:
datanumber=nbofdata
blob=[]
blob_nparray=[]
image_data=[]
conc = 0
fc = 0
labels_dict={}
fn = {}
for entry1 in folder_list[layers_of_folders - 1]:
    blob = []
    cellname = os.path.basename(os.path.dirname(entry1))  # extract cell name
    # print(cellname)
    concnames = os.path.basename(entry1)  # extract concentration
    # print(concnames)
    if concnames in labels:
        labels_dict[conc] = concnames
        fnamelist = glob.glob(os.path.join(entry1, '*.jpg'))
        for filename in fnamelist[0:datanumber]:
            im = Image.open(filename)
            if im is not None:
                if im.mode=='RGB':
                    im=im.resize(size,Image.BILINEAR)
                    imarray = np.array(im)
                    blob.append(imarray)
                    fn[fc] = filename
                    fc += 1
        ind = np.reshape(np.arange(1, len(blob) + 1), (-1, 1))
        blob_nparray = np.reshape(np.asarray(blob), (len(blob), blob[1].size))
        blob_nparray = np.hstack((blob_nparray, ind, conc * np.ones((len(blob), 1))))
        image_data.append(np.asarray(blob_nparray, dtype=np.float32))
        print(ind)
        print(blob_nparray)
        print(concnames+'  finished!')
        conc += 1

[[   1]
 [   2]
 [   3]
 ...
 [1998]
 [1999]
 [2000]]
[[1.150e+02 1.050e+02 9.000e+00 ... 3.000e+00 1.000e+00 0.000e+00]
 [1.970e+02 2.150e+02 2.240e+02 ... 4.800e+01 2.000e+00 0.000e+00]
 [3.500e+01 6.100e+01 3.600e+01 ... 3.200e+01 3.000e+00 0.000e+00]
 ...
 [1.080e+02 1.230e+02 3.000e+01 ... 0.000e+00 1.998e+03 0.000e+00]
 [1.390e+02 1.720e+02 1.990e+02 ... 1.000e+00 1.999e+03 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 ... 1.000e+00 2.000e+03 0.000e+00]]
test  finished!


In [4]:
for j in range(len(labels)):
    trytry=image_data[j][:]
# Prepare data
    LengthT = trytry.shape[0]

    trytry_index = trytry[...,-2:-1]

    trytry_label = trytry[...,-1:] #['Nega' for x in range(lengthN*4)] #Nega_data[...,-1:]

    trytry = trytry[...,:-2]

    # Normalize image by subtracting mean image
    trytry -= np.reshape(np.mean(trytry, axis=1), (-1,1))
    # Reshape images
    trytry = np.reshape(trytry, (trytry.shape[0],128,128,3))
    
#    # Rotate images
#    for i in range(3):
#        trytry[LengthT*(i+1):LengthT*(i+2)] = np.rot90(trytry[:LengthT], i+1, (1,2))
    # Add channel dimension to fit in Conv2D
    trytry = trytry.reshape(-1,128,128,3)
    trytry_test_upto = trytry.shape[0]
    if j is 0:
        test_data = trytry[:trytry_test_upto]      
    else:     
        test_data = np.concatenate((test_data, 
                                    trytry[trytry_train_upto:trytry_test_upto]), axis=0)

In [5]:
sp = {}
for i in range(2000):
    sp[i] = fn[i].split('/')
    sp[i] = sp[i][4].split('.jpg')

In [6]:
ids = {}
for i in range(2000):
    ids[i] = sp[i][0]

In [7]:
from keras.models import load_model
model = load_model("./flower_model.h5")

In [8]:
predictions=model.predict(test_data)

In [9]:
for i in range(10):
    for j in range(5):
        print(predictions[i][j])

3.9935112e-05
0.00012457371
0.0127028525
0.9983238
0.0001937151
0.98664266
0.011807382
0.0291273
0.015110195
0.026175797
0.007658154
0.0020169914
0.9679964
0.043725938
0.027048737
0.9974655
0.0032157898
0.008302808
0.00070914626
0.011686325
0.22134948
0.03497374
0.34177154
0.36997214
0.07343039
0.0002478063
0.00013196468
0.00891611
0.998193
7.376075e-05
0.9637177
0.012692094
0.05468023
0.012762368
0.036421895
0.11166185
0.07628694
0.114750296
0.12957805
0.7082256
0.011963159
0.0047141016
0.977321
0.03268215
0.011747748
0.08070183
0.6545901
0.102231055
0.12283537
0.29283065


In [10]:
cnn_pred = {}
for i in range(2000):
    for j in range(5):
        if predictions[i][j] == np.max(predictions[i]):
            if j == 0:
                cnn_pred[i] = 3
            if j == 1:
                cnn_pred[i] = 4
            if j == 2:
                cnn_pred[i] = 1
            if j == 3:
                cnn_pred[i] = 0
            if j == 4:
                cnn_pred[i] = 2

In [11]:
cnn_pred

{0: 0,
 1: 3,
 2: 1,
 3: 3,
 4: 0,
 5: 0,
 6: 3,
 7: 2,
 8: 1,
 9: 4,
 10: 4,
 11: 2,
 12: 4,
 13: 1,
 14: 0,
 15: 2,
 16: 3,
 17: 0,
 18: 1,
 19: 0,
 20: 1,
 21: 0,
 22: 4,
 23: 4,
 24: 3,
 25: 0,
 26: 1,
 27: 3,
 28: 0,
 29: 0,
 30: 3,
 31: 3,
 32: 2,
 33: 3,
 34: 2,
 35: 0,
 36: 4,
 37: 2,
 38: 1,
 39: 1,
 40: 2,
 41: 0,
 42: 1,
 43: 4,
 44: 3,
 45: 1,
 46: 1,
 47: 0,
 48: 0,
 49: 1,
 50: 0,
 51: 0,
 52: 0,
 53: 0,
 54: 1,
 55: 1,
 56: 4,
 57: 0,
 58: 1,
 59: 2,
 60: 3,
 61: 0,
 62: 0,
 63: 1,
 64: 1,
 65: 0,
 66: 1,
 67: 3,
 68: 1,
 69: 0,
 70: 1,
 71: 0,
 72: 0,
 73: 0,
 74: 0,
 75: 2,
 76: 2,
 77: 0,
 78: 1,
 79: 2,
 80: 1,
 81: 0,
 82: 2,
 83: 3,
 84: 0,
 85: 1,
 86: 0,
 87: 1,
 88: 2,
 89: 0,
 90: 3,
 91: 2,
 92: 2,
 93: 1,
 94: 3,
 95: 1,
 96: 2,
 97: 3,
 98: 0,
 99: 0,
 100: 3,
 101: 1,
 102: 0,
 103: 2,
 104: 1,
 105: 3,
 106: 3,
 107: 0,
 108: 0,
 109: 3,
 110: 1,
 111: 1,
 112: 1,
 113: 1,
 114: 0,
 115: 0,
 116: 1,
 117: 1,
 118: 2,
 119: 0,
 120: 1,
 121: 1,
 122: 2,
 12

In [12]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

submit = pd.DataFrame({'id': ids, 'flower_class': cnn_pred})
header = ["id", "flower_class"]
submit.to_csv('cnn_predict_lc.csv', columns = header, index=False)